In [1]:
from owlready2 import *
#import types
from conllu import parse_incr, parse
import numpy as np
import pandas as pd

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Carregando ontologias

In [2]:
#OntoGeoLógica populada (OntoGeoLogicaPovoadaInstanciasRelacoes.owl)
#onto_pop = get_ontology("..\OntoGeoLogicaInstanciasRelacoes.owl")
#onto_pop.load()
onto = get_ontology("../OntoGeoLogicaEntidadesNomeadas.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

Carregando Listas de entidedes

In [3]:
#Completo
#text_treino = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - sentences.npy')
entities_PetroNER = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - entities.npy')
classes_PetroNER = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - classes.npy')
URI_PetroNER = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - URI.npy', allow_pickle=True)

#Treino
#text_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - sentences.npy')
entities_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - entities.npy')
classes_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - classes.npy')
URI_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - URI.npy', allow_pickle=True)

#Valid
#text_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - sentences.npy')
entities_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - entities.npy')
classes_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - classes.npy')
URI_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - URI.npy', allow_pickle=True)

#Teste
#text_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - sentences.npy')
entities_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - entities.npy')
classes_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - classes.npy')
URI_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - URI.npy', allow_pickle=True)

Carregando arquivo com relações

In [4]:
# df_filtred = pickle.load(open('../../Corpora/PetroRE/df_relation.pkl', 'rb'))
df_relations = pd.read_csv('../../Corpora/PetroRE/df_relation.csv')

### Verificando se tem URI com duas classes diferentes

In [5]:
#Coletando tuplas com clases e URIs
entity_tupla = []

for n in range(len(URI_PetroNER)):
    entity_tupla.append((URI_PetroNER[n],classes_PetroNER[n]))

#Set de tuplas
set_tupla = set(entity_tupla)

# Excluido as URIs None
ent = []
for tup in set_tupla:
    #print(tup[0])
    if tup[0] != None:
        ent.append(tup[0])
        
#Ordenando as URIs e imprimindo quando tem uma URI repetida
classes_duplicadas = []
ent.sort()
ant = ''
for i in ent:
    if i == ant:
        print(i)
        classes_duplicadas.append(i)
    ant = i

#Dom_Joao_Age
#TEFR_CD_TIPO_EST_FISICA_ROCHA_059
#earth_fluid
#fluid_composition
#fluid_origin
#water
#water


### Povoamento a ontologia com as entidades

Separando as URI por classes

In [6]:
# Função que recebe as listas com todas URIs e Classes, 
# e retorna apenas um set das URI doselementos da classe indicada em 'string_classe', 
# com exceção da URI da própria classe (indicado em URI_classe)

def lista_por_calsse(URI_dataset, classes_dataset, string_classe, URI_classe):

    lista_classe = []
        
    #Iterando por todos os elementos
    for n in range(len(URI_dataset)):
        #Verificando qual a classe
        if ((classes_dataset[n] == string_classe) and 
            (URI_dataset[n]!= URI_classe) and 
            (URI_dataset[n]!= None) and
            (URI_dataset[n]not in classes_duplicadas)):
            lista_classe.append(URI_dataset[n])


    # Reduzindo a lista para apenas elementos únicos
    lista_classe = set(lista_classe)
    
    return lista_classe

In [7]:
BACIA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'BACIA', '#basin')
CAMPO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'CAMPO', '#field')
ESTRUTURA_FÍSICA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'ESTRUTURA_FÍSICA', '#geological_structure')
EVENTO_PETRO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'EVENTO_PETRO', '#geological_process')
FLUIDO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'FLUIDO', '#anthropogenic_fluid')
FLUIDODATERRA_i_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'FLUIDODATERRA_i', '#o&g_earth_fluid')
FLUIDODATERRA_o_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'FLUIDODATERRA_o', '#earth_fluid')
NÃOCONSOLID_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'NÃOCONSOLID', '#unconsolidated_earth_material')
POÇO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'POÇO', '#well')
ROCHA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'ROCHA', '#rock')
TEXTURA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'TEXTURA', '#textura')
UNIDADE_CRONO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'UNIDADE_CRONO', '#geological_age')
UNIDADE_LITO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'UNIDADE_LITO', '#lithostratigraphic_unit')

Povoando as subclasses

In [8]:
#Função que recebe a classe da ontologia e a lista de sublasses para povoar
def povoando_subclasses(onto, classe, lista_classe):

    for new_class in lista_classe:
        with onto:
            types.new_class(new_class[1:], (onto[classe],))
    return

In [9]:
povoando_subclasses(onto, 'rock', ROCHA_list)
povoando_subclasses(onto, 'unconsolidated_earth_material', NÃOCONSOLID_list)
povoando_subclasses(onto, 'o&g_earth_fluid', FLUIDODATERRA_i_list)
povoando_subclasses(onto, 'earth_fluid', FLUIDODATERRA_o_list)

Povoando as instâcias

In [10]:
for basin in BACIA_list:
    onto['basin'](basin[1:])

for field in CAMPO_list:
    onto['field'](field[1:])
    
for structure in ESTRUTURA_FÍSICA_list:
    onto['geological_structure'](structure[1:])
    
for process in EVENTO_PETRO_list:
    onto['geological_process'](process[1:])
    
for well in POÇO_list:
    onto['well'](well[1:])

for textura in TEXTURA_list:
    onto['textura'](textura[1:])

for lithostratigraphic_unit in UNIDADE_LITO_list:
    #if lithostratigraphic_unit != '#Dom_Joao_Age':
    onto['lithostratigraphic_unit'](lithostratigraphic_unit[1:])
    

Trantando o caso específico da Unidade Cronoestratigráfica.  
  
  Para cada URI anotada no corpus, povoamos na ontologia um subclasse de onto.geological_age, terminando com '_quality' e uma instância de onto.geological_time_interval com a URI anotada no corpus.

In [11]:
for crono in UNIDADE_CRONO_list:
    with onto:
        #povoando as classes geological age e geological time interval
        types.new_class(crono[1:] + '_quality', (onto['geological_age'],))
        onto['geological_time_interval'](crono[1:])

### Povoando a ontologia com as relações

In [12]:
df_relations

,Relation,Ent1,Ent2,URI_1,URI_2,#Sentence
0,located_in,UNIDADE_LITO,BACIA,#membro_010,#BASE_CD_BACIA_270,6
1,located_in,UNIDADE_LITO,BACIA,#formacao_319,#BASE_CD_BACIA_100,97
2,located_in,UNIDADE_LITO,BACIA,#formacao_319,#BASE_CD_BACIA_100,97
3,located_in,UNIDADE_LITO,BACIA,#formacao_319,#BASE_CD_BACIA_100,97
4,located_in,UNIDADE_LITO,BACIA,#formacao_319,#BASE_CD_BACIA_100,97
...,...,...,...,...,...,...
2287,temporal_relation,UNIDADE_CRONO,UNIDADE_CRONO,#Barremian,#Aptian,23968
2288,temporal_relation,UNIDADE_CRONO,UNIDADE_CRONO,#Aptian,#Barremian,23968
2289,located_in,UNIDADE_LITO,BACIA,#formacao_222,#BASE_CD_BACIA_281,23988
2290,temporal_relation,UNIDADE_CRONO,UNIDADE_CRONO,#Valanginian,#LowerCretaceous,23994


In [13]:
# Colentando as triplas (URI_1, Relation, URI_2)
triplas = list(df_relations.apply(lambda x: (x['URI_1'], x['Relation'], x['URI_2']), axis=1))
print('Número de triplas:', len(triplas))
triplas = set(triplas)
print('Número de triplas únicas:', len(triplas))

Número de triplas: 2292
Número de triplas únicas: 899


In [14]:
set(list(df_relations['Relation']))

{'constituted_by',
 'crosses',
 'has_age',
 'interval_finishes',
 'located_in',
 'temporal_relation'}

In [15]:
for tripla in triplas:
    
    if ((tripla[0] not in classes_duplicadas) and 
        (tripla[2] not in classes_duplicadas)):
    
        if tripla[1] == 'constituted_by':
            onto[tripla[0][1:]].constituted_by.append(onto[tripla[2][1:]])

        if tripla[1] == 'crosses':
            onto[tripla[0][1:]].crosses.append(onto[tripla[2][1:]])

        if tripla[1] == 'has_age':
            # Criando a instância "idade geológica da unidade lito"
            onto[tripla[2][1:] + '_quality'](tripla[2][1:] + '_quality_' + tripla[0][1:])
            
            # Criando a relação entra a unidade lito e instância "idade geológica da unidade lito"
            onto[tripla[0][1:]].has_age.append(onto[tripla[2][1:] + '_quality_' + tripla[0][1:]])
            
            # Adicionando a relação partipates_in entre a qualidade Geological age e o Geological Time interval
  
            onto[tripla[2][1:] + '_quality_' + tripla[0][1:]].participates_in.append(onto[tripla[2][1:]])
            
        if tripla[1] == 'located_in':
            onto[tripla[0][1:]].located_in.append(onto[tripla[2][1:]])
                
        if tripla[1] == 'temporal_relation':
            onto[tripla[0][1:]].has_temporal_relation.append(onto[tripla[2][1:]])
            

### Estatísticas

In [ ]:
BACIA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'BACIA', '#basin')
CAMPO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'CAMPO', '#field')
ESTRUTURA_FÍSICA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'ESTRUTURA_FÍSICA', '#geological_structure')
EVENTO_PETRO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'EVENTO_PETRO', '#geological_process')
FLUIDO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'FLUIDO', '#anthropogenic_fluid')
FLUIDODATERRA_i_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'FLUIDODATERRA_i', '#o&g_earth_fluid')
FLUIDODATERRA_o_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'FLUIDODATERRA_o', '#earth_fluid')
NÃOCONSOLID_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'NÃOCONSOLID', '#unconsolidated_earth_material')
POÇO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'POÇO', '#well')
ROCHA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'ROCHA', '#rock')
TEXTURA_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'TEXTURA', '#textura')
UNIDADE_CRONO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'UNIDADE_CRONO', '#geological_age')
UNIDADE_LITO_list = lista_por_calsse(URI_PetroNER, classes_PetroNER, 'UNIDADE_LITO', '#lithostratigraphic_unit')

In [17]:
print('Total individuals: ', len(list(onto.individuals())))
print('Bacias: ', len(onto.search(type = onto.basin)))
print('Campo: ', len(onto.search(type = onto.field)))
print('Estrutura Física: ', len(onto.search(type = onto.geological_structure)))
print('Poço: ', len(onto.search(type = onto.well)))
print('Textura: ', len(onto.search(type = onto.textura)))
print('Unidade Lito: ', len(onto.search(type = onto.lithostratigraphic_unit)))
print('Intervalo de tempo Geológico: ', len(onto.search(type = onto.geological_time_interval)))
print('Idade Geologica: ', len(onto.search(type = onto.geological_age)))

Total individuals:  904
Bacias:  65
Campo:  110
Estrutura Física:  62
Poço:  193
Textura:  19
Unidade Lito:  212
Intervalo de tempo Geológico:  131
Idade Geologica:  112


In [22]:
print('Total classes: ', len(list(onto.classes())))
print('Fluido antropogênico: ', len(onto.search(subclass_of = onto['anthropogenic_fluid'])))
print('Fluido da terra O&G: ', len(onto.search(subclass_of = onto['o&g_earth_fluid'])))
print('Fluido da terra: ', len(onto.search(subclass_of = onto['earth_fluid'])))
print('Material não consolidado: ', len(onto.search(subclass_of = onto['unconsolidated_earth_material'])))
print('Rocha: ', len(onto.search(subclass_of = onto['rock'])))

Total classes:  259
Fluido antropogênico:  1
Fluido da terra O&G:  7
Fluido da terra:  10
Material não consolidado:  11
Rocha:  79


In [61]:
print('constituted_by : ', len(list(onto.constituted_by.get_relations())))
print('crosses : ', len(list(onto.crosses.get_relations())))
print('has_age : ', len(list(onto.has_age.get_relations())))
print('located_in : ', len(list(onto.located_in.get_relations())))
print('temporal_relation : ', len(list(onto.has_temporal_relation.get_relations())))

constituted_by :  204
crosses :  25
has_age :  112
located_in :  239
temporal_relation :  310
